In [1]:
!pip install py2neo


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from py2neo import Graph

# Conectarse a Neo4j
graph = Graph("bolt://neo4j:7687", auth=("neo4j", "password"))

# Ejecutar una consulta de Cypher usando una cadena de múltiples líneas
query = """
MATCH (a:Artist)
RETURN a.name AS artist_name, a.popularity AS artist_popularity
ORDER BY artist_popularity DESC
LIMIT 5
"""
nodes = graph.run(query).data()

# Mostrar los resultados
for node in nodes:
    print(node)


{'artist_name': 'Bad Bunny', 'artist_popularity': 92}
{'artist_name': 'The Weeknd', 'artist_popularity': 92}
{'artist_name': 'Travis Scott', 'artist_popularity': 91}
{'artist_name': 'Eminem', 'artist_popularity': 87}
{'artist_name': 'David Guetta', 'artist_popularity': 86}
